In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../")
import os

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch
from nnsight import LanguageModel
from src.utils import env_utils

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:15<00:00,  2.52s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [4]:
from src.dataset import load_worldstate_dataset
from src.functional import predict_next_token 

dataset = load_worldstate_dataset()
q1, a1 = dataset.__getitem__(5, tags=["a", "b"])
print(q1)
print(a1)

predict_next_token(
    lm=lm,
    input=q1
)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option (either a or b) after the "Answer:" tag.

Story: Ravi is a farmer in a small Indian village, tending to his mango orchard. Ravi wants to harvest ripe mangoes to sell at the local market. He notices a mango tree with a heavy branch laden with ripe mangoes. A mischievous monkey comes along and eats all the ripe mangoes on the branch while Ravi is gathering his harvesting tools. Ravi sees the monkey eating the ripe mangoes.

Question: Does the branch have ripe mangoes or no ripe mangoes?
a) The branch has no ripe mangoes.
b) The branch has ripe mangoes.
Answer:
a


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


[PredictedToken(token=' a', prob=0.9979391694068909, logit=23.640625, token_id=264),
 PredictedToken(token=' b', prob=0.0007783666369505227, logit=16.484375, token_id=293),
 PredictedToken(token=' ', prob=0.00035083730472251773, logit=15.6875, token_id=220),
 PredictedToken(token=' (', prob=0.00018487827037461102, logit=15.046875, token_id=320),
 PredictedToken(token=' The', prob=0.00015207672549877316, logit=14.8515625, token_id=578)]

In [5]:
q2, a2 = dataset.__getitem__(15, tags=["x", "y"])
print(q2)
print(a2)

predict_next_token(
    lm=lm,
    input=q2
)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option (either x or y) after the "Answer:" tag.

Story: Mei is a florist in a small town in Japan. She wants to create a beautiful bouquet using fresh flowers from her garden. Mei notices that her roses are in full bloom, with vibrant colors. Overnight, a sudden frost damages the roses, causing them to wilt and lose their vibrancy. Mei sees the wilted roses in her garden the next morning.

Question: Are the roses in Mei's garden in full bloom or wilted?
x) The roses are in full bloom.
y) The roses are wilted.
Answer:
y


[PredictedToken(token=' y', prob=0.9976453185081482, logit=22.5, token_id=379),
 PredictedToken(token=' **', prob=0.00027102624881081283, logit=14.2890625, token_id=3146),
 PredictedToken(token=' __', prob=0.00025861503672786057, logit=14.2421875, token_id=1328),
 PredictedToken(token=' \n', prob=0.000256602477747947, logit=14.234375, token_id=720),
 PredictedToken(token=' The', prob=0.0002546055766288191, logit=14.2265625, token_id=578)]

In [7]:
from src.functional import get_hs
from src.functional import logit_lens
from src.functional import free_gpu_cache

layer_name_format = "model.layers.{}"
free_gpu_cache()

hs = get_hs(
    lm=lm,
    input=q1,
    locations=[
        (layer_name_format.format(l), -1) 
        for l in range(lm.config.num_hidden_layers)
    ]
)

free_gpu_cache()

OutOfMemoryError: CUDA out of memory. Tried to allocate 896.00 MiB. GPU 

In [107]:
hs.keys()

dict_keys([('model.layers.0', -1), ('model.layers.1', -1), ('model.layers.2', -1), ('model.layers.3', -1), ('model.layers.4', -1), ('model.layers.5', -1), ('model.layers.6', -1), ('model.layers.7', -1), ('model.layers.8', -1), ('model.layers.9', -1), ('model.layers.10', -1), ('model.layers.11', -1), ('model.layers.12', -1), ('model.layers.13', -1), ('model.layers.14', -1), ('model.layers.15', -1), ('model.layers.16', -1), ('model.layers.17', -1), ('model.layers.18', -1), ('model.layers.19', -1), ('model.layers.20', -1), ('model.layers.21', -1), ('model.layers.22', -1), ('model.layers.23', -1), ('model.layers.24', -1), ('model.layers.25', -1), ('model.layers.26', -1), ('model.layers.27', -1), ('model.layers.28', -1), ('model.layers.29', -1), ('model.layers.30', -1), ('model.layers.31', -1)])

In [110]:
from src.functional import PatchSpec

patches = [
    PatchSpec(
        location=key,
        patch=hs[key]
    ) for key in hs.keys()
]

predict_next_token(
    lm=lm, 
    input=q2,
    patches=patches
)

[PredictedToken(token=' __________________', prob=0.3442102074623108, logit=16.4375, token_id=44941),
 PredictedToken(token=' ______', prob=0.23290430009365082, logit=16.046875, token_id=33771),
 PredictedToken(token=' \n', prob=0.14574778079986572, logit=15.578125, token_id=720),
 PredictedToken(token=' __', prob=0.040157731622457504, logit=14.2890625, token_id=1328),
 PredictedToken(token=' __________________________________', prob=0.03656403347849846, logit=14.1953125, token_id=82108)]

In [9]:
from tqdm.auto import tqdm
import random

tag_options = [
    ("a", "b"),
    ("x", "y"), 
    ("p", "q"),
]

progress = tqdm(range(len(dataset)))
correct = 0
for i in progress:
    question, answer = dataset.__getitem__(i, tags=random.choice(tag_options))
    lm_ans = get_lm_answer(lm=lm, question=question)
    correct += lm_ans.strip() == answer.strip()
    # print(f"expected: {answer.strip()} | got: {lm_ans.strip()}")
    progress.set_description(f"{i+1}/{len(dataset)} => acc: {correct/(i+1)}({correct=})")

200/200 => acc: 0.99(correct=198): 100%|██████████| 200/200 [10:01<00:00,  3.01s/it]              
